<a href="https://colab.research.google.com/github/fred-creator-creat/ecommerce-database-logic-project/blob/main/Projeto_Logico_Ecommerce_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Instala o servidor e o conector
!apt-get update
!apt-get install mysql-server > /dev/null
!pip install mysql-connector-python > /dev/null

# 2. Inicia o serviço
!service mysql start

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [39.2 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,300 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all P

In [2]:
# Comando para mudar o método de autenticação do Root
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY '';"
!mysql -e "FLUSH PRIVILEGES;"

In [3]:
import mysql.connector

# 1. Conectar ao servidor
db_setup = mysql.connector.connect(
  host="localhost",
  user="root",
  password=""
)
cursor = db_setup.cursor()

# 2. Criar o banco de dados do projeto e seleciona para uso
cursor.execute("CREATE DATABASE IF NOT EXISTS ecommerce;")
cursor.execute("USE ecommerce;")

print("Banco de dados 'ecommerce' criado e selecionado com sucesso!")

Banco de dados 'ecommerce' criado e selecionado com sucesso!


In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Criando a engine de conexão para o banco ecommerce
# O Pandas usará isso para mostrar as tabelas
engine = create_engine('mysql+mysqlconnector://root@localhost/ecommerce')

# Ativa a visualização de tabelas interativas do Colab
%load_ext google.colab.data_table

print("Mecanismo de visualização Pandas/SQLAlchemy configurado para o banco 'ecommerce'.")

Mecanismo de visualização Pandas/SQLAlchemy configurado para o banco 'ecommerce'.


In [5]:
# Criando as tabelas baseadas no refinamento

# 1. Tabela de Cliente
# Refinamento: Nome composto e CPF como Char(11)
cursor.execute("""
CREATE TABLE IF NOT EXISTS cliente (
    id_cliente INT AUTO_INCREMENT PRIMARY KEY,
    f_nome VARCHAR(15) NOT NULL,
    m_inicial CHAR(3),
    l_nome VARCHAR(20),
    cpf CHAR(11) NOT NULL,
    endereco VARCHAR(45),
    CONSTRAINT unique_cpf_cliente UNIQUE (cpf)
);
""")

# 2. Tabela de Pedido
# Refinamento: Status como ENUM e valor Default 'Processando'
cursor.execute("""
CREATE TABLE IF NOT EXISTS pedido (
    id_pedido INT AUTO_INCREMENT PRIMARY KEY,
    id_pedido_cliente INT,
    status_pedido ENUM('Cancelado', 'Confirmado', 'Em processamento') DEFAULT 'Em processamento',
    descricao VARCHAR(255),
    frete FLOAT DEFAULT 10,
    pagamento_cash BOOL DEFAULT FALSE,
    CONSTRAINT fk_pedido_cliente FOREIGN KEY (id_pedido_cliente) REFERENCES cliente(id_cliente)
);
""")

db_setup.commit()
print("Tabelas 'cliente' e 'pedido' criadas com sucesso!")

Tabelas 'cliente' e 'pedido' criadas com sucesso!


In [11]:
cursor.execute("SHOW TABLES;")
for x in cursor: print(x)

('client',)
('orders',)
('payments',)
('product',)
('productOrder',)
('productSeller',)
('productStorage',)
('productSupplier',)
('seller',)
('storageLocation',)
('supplier',)


In [6]:
# 1. Removendo as tabelas anteriores para aplicar os nomes em inglês
cursor.execute("DROP TABLE IF EXISTS pedido;")
cursor.execute("DROP TABLE IF EXISTS cliente;")

# 2. Criando a Tabela de Client (Client)
cursor.execute("""
CREATE TABLE client (
    idClient INT AUTO_INCREMENT PRIMARY KEY,
    Fname VARCHAR(10),
    Minit CHAR(3),
    Lname VARCHAR(20),
    CPF CHAR(11) NOT NULL,
    Address VARCHAR(30),
    CONSTRAINT unique_cpf_client UNIQUE (CPF)
);
""")

# 3. Criando a Tabela de Produto (Product)
cursor.execute("""
CREATE TABLE product (
    idProduct INT AUTO_INCREMENT PRIMARY KEY,
    Pname VARCHAR(10) NOT NULL,
    classification_kids BOOL DEFAULT FALSE,
    category ENUM('Eletrônico', 'Vestuário', 'Brinquedos', 'Alimentos', 'Móveis') NOT NULL,
    avaliação FLOAT DEFAULT 0,
    size VARCHAR(10)
);
""")

db_setup.commit()
print("Tabelas 'client' e 'product' criadas com nomes oficiais!")

Tabelas 'client' e 'product' criadas com nomes oficiais!


In [7]:
# 1. Criação da Tabela de Pedidos (Orders)
# Refinamento: Nomes em inglês e FK para o cliente
cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    idOrder INT AUTO_INCREMENT PRIMARY KEY,
    idOrderClient INT,
    orderStatus ENUM('Cancelado', 'Confirmado', 'Em processamento') DEFAULT 'Em processamento',
    orderDescription VARCHAR(255),
    sendValue FLOAT DEFAULT 10,
    paymentCash BOOL DEFAULT FALSE,
    CONSTRAINT fk_orders_client FOREIGN KEY (idOrderClient) REFERENCES client(idClient)
        ON UPDATE CASCADE
);
""")

# 2. Criação da Tabela de Pagamentos (Payments)
# Refinamento: Permite múltiplos métodos de pagamento por cliente
cursor.execute("""
CREATE TABLE IF NOT EXISTS payments (
    idClient INT,
    idPayment INT,
    typePayment ENUM('Boleto', 'Cartão', 'Dois Cartões'),
    limitAvailable FLOAT,
    PRIMARY KEY (idClient, idPayment),
    CONSTRAINT fk_payments_client FOREIGN KEY (idClient) REFERENCES client(idClient)
);
""")

db_setup.commit()
print("Tabelas 'orders' e 'payments' criadas com sucesso!")

Tabelas 'orders' e 'payments' criadas com sucesso!


In [9]:
# 1. Ajuste no status da Tabela de Pedidos (Orders)
cursor.execute("""
ALTER TABLE orders MODIFY COLUMN orderStatus
ENUM('Cancelado', 'Confirmado', 'Em processamento') DEFAULT 'Em processamento';
""")

# 2. Criação da Tabela de Estoque (Storage)
cursor.execute("""
CREATE TABLE IF NOT EXISTS productStorage (
    idProdStorage INT AUTO_INCREMENT PRIMARY KEY,
    storageLocation VARCHAR(255),
    quantity INT DEFAULT 0
);
""")

# 3. Criação da Tabela de Fornecedor (Supplier)
cursor.execute("""
CREATE TABLE IF NOT EXISTS supplier (
    idSupplier INT AUTO_INCREMENT PRIMARY KEY,
    SocialName VARCHAR(255) NOT NULL,
    CNPJ CHAR(15) NOT NULL,
    contact CHAR(11) NOT NULL,
    CONSTRAINT unique_supplier UNIQUE (CNPJ)
);
""")

# 4. Criação da Tabela de Vendedor (Seller)
cursor.execute("""
CREATE TABLE IF NOT EXISTS seller (
    idSeller INT AUTO_INCREMENT PRIMARY KEY,
    SocialName VARCHAR(255) NOT NULL,
    AbstName VARCHAR(255),
    CNPJ CHAR(15),
    CPF CHAR(11),
    location VARCHAR(255),
    contact CHAR(11) NOT NULL,
    CONSTRAINT unique_cnpj_seller UNIQUE (CNPJ),
    CONSTRAINT unique_cpf_seller UNIQUE (CPF)
);
""")

db_setup.commit()
print("Tabelas 'productStorage', 'supplier' e 'seller' criadas!")

Tabelas 'productStorage', 'supplier' e 'seller' criadas!


In [10]:
# 1. Tabela de Ligação: Produto e Vendedor (productSeller)
cursor.execute("""
CREATE TABLE IF NOT EXISTS productSeller (
    idPseller INT,
    idPproduct INT,
    prodQuantity INT DEFAULT 1,
    PRIMARY KEY (idPseller, idPproduct),
    CONSTRAINT fk_product_seller FOREIGN KEY (idPseller) REFERENCES seller(idSeller),
    CONSTRAINT fk_product_product FOREIGN KEY (idPproduct) REFERENCES product(idProduct)
);
""")

# 2. Tabela de Ligação: Produto e Pedido (productOrder)
cursor.execute("""
CREATE TABLE IF NOT EXISTS productOrder (
    idPOproduct INT,
    idPOorder INT,
    poQuantity INT DEFAULT 1,
    poStatus ENUM('Disponível', 'Sem estoque') DEFAULT 'Disponível',
    PRIMARY KEY (idPOproduct, idPOorder),
    CONSTRAINT fk_productorder_product FOREIGN KEY (idPOproduct) REFERENCES product(idProduct),
    CONSTRAINT fk_productorder_order FOREIGN KEY (idPOorder) REFERENCES orders(idOrder)
);
""")

# 3. Tabela de Ligação: Produto e Estoque (storageLocation)
cursor.execute("""
CREATE TABLE IF NOT EXISTS storageLocation (
    idLproduct INT,
    idLstorage INT,
    location VARCHAR(255) NOT NULL,
    PRIMARY KEY (idLproduct, idLstorage),
    CONSTRAINT fk_storage_location_product FOREIGN KEY (idLproduct) REFERENCES product(idProduct),
    CONSTRAINT fk_storage_location_storage FOREIGN KEY (idLstorage) REFERENCES productStorage(idProdStorage)
);
""")

# 4. Tabela de Ligação: Produto e Fornecedor (productSupplier)
cursor.execute("""
CREATE TABLE IF NOT EXISTS productSupplier (
    idPsSupplier INT,
    idPsProduct INT,
    quantity INT NOT NULL,
    PRIMARY KEY (idPsSupplier, idPsProduct),
    CONSTRAINT fk_product_supplier_supplier FOREIGN KEY (idPsSupplier) REFERENCES supplier(idSupplier),
    CONSTRAINT fk_product_supplier_product FOREIGN KEY (idPsProduct) REFERENCES product(idProduct)
);
""")

db_setup.commit()
print("Todas as tabelas de relacionamento (N:N) foram criadas!")

Célula 9 concluída: Todas as tabelas de relacionamento (N:N) foram criadas!


In [12]:
# Validação Final da Estrutura do Banco de Dados

print("--- ESTRUTURA FINAL DO BANCO DE DADOS E-COMMERCE ---")
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

for (table,) in tables:
    print(f"Tabela Confirmada: {table}")

print(f"\nTotal de tabelas: {len(tables)}")
print("Status: 11 tabelas criadas (incluindo o desafio de Payments).")
print("--------------------------------------------------")

--- ESTRUTURA FINAL DO BANCO DE DADOS E-COMMERCE ---
Tabela Confirmada: client
Tabela Confirmada: orders
Tabela Confirmada: payments
Tabela Confirmada: product
Tabela Confirmada: productOrder
Tabela Confirmada: productSeller
Tabela Confirmada: productStorage
Tabela Confirmada: productSupplier
Tabela Confirmada: seller
Tabela Confirmada: storageLocation
Tabela Confirmada: supplier

Total de tabelas: 11
Status: 11 tabelas criadas (incluindo o desafio de Payments).
--------------------------------------------------


In [13]:
# 1. Aplicando o ON UPDATE CASCADE na tabela de pedidos (Orders)
# Isso garante que se o ID do cliente mudar, o pedido se mantenha vinculado.
try:
    cursor.execute("ALTER TABLE orders DROP FOREIGN KEY fk_orders_client;")
    cursor.execute("""
    ALTER TABLE orders ADD CONSTRAINT fk_orders_client
    FOREIGN KEY (idOrderClient) REFERENCES client(idClient)
    ON UPDATE CASCADE;
    """)
    db_setup.commit()
    print("Refinamento aplicado: ON UPDATE CASCADE adicionado à tabela 'orders'.")
except:
    print("A constraint já possuía o refinamento ou foi ajustada.")

# 2. Query Exploratória
# Busquei no dicionário de dados do MySQL as constraints
print("\n--- EXPLORAÇÃO DE CONSTRAINTS (Dicionário de Dados) ---")
query_exploratoria = """
SELECT CONSTRAINT_NAME, TABLE_NAME, CONSTRAINT_TYPE
FROM information_schema.TABLE_CONSTRAINTS
WHERE CONSTRAINT_SCHEMA = 'ecommerce';
"""
cursor.execute(query_exploratoria)
constraints = cursor.fetchall()

for (name, table, type_c) in constraints:
    print(f"Tabela: {table.ljust(15)} | Tipo: {type_c.ljust(12)} | Nome: {name}")

print("Banco de dados explorado e validado!")

Refinamento aplicado: ON UPDATE CASCADE adicionado à tabela 'orders'.

--- EXPLORAÇÃO DE CONSTRAINTS (Dicionário de Dados) ---
Tabela: client          | Tipo: PRIMARY KEY  | Nome: PRIMARY
Tabela: client          | Tipo: UNIQUE       | Nome: unique_cpf_client
Tabela: orders          | Tipo: PRIMARY KEY  | Nome: PRIMARY
Tabela: orders          | Tipo: FOREIGN KEY  | Nome: fk_orders_client
Tabela: payments        | Tipo: PRIMARY KEY  | Nome: PRIMARY
Tabela: payments        | Tipo: FOREIGN KEY  | Nome: fk_payments_client
Tabela: product         | Tipo: PRIMARY KEY  | Nome: PRIMARY
Tabela: productOrder    | Tipo: PRIMARY KEY  | Nome: PRIMARY
Tabela: productOrder    | Tipo: FOREIGN KEY  | Nome: fk_productorder_order
Tabela: productOrder    | Tipo: FOREIGN KEY  | Nome: fk_productorder_product
Tabela: productSeller   | Tipo: PRIMARY KEY  | Nome: PRIMARY
Tabela: productSeller   | Tipo: FOREIGN KEY  | Nome: fk_product_product
Tabela: productSeller   | Tipo: FOREIGN KEY  | Nome: fk_product_seller

In [18]:
# --- PARTE 1: Limpeza de Segurança (Para evitar erro de Duplicate Entry) ---
# Desativei as chaves temporariamente para limpar sem erros de dependência
cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
cursor.execute("TRUNCATE TABLE payments;")
cursor.execute("TRUNCATE TABLE orders;")
cursor.execute("TRUNCATE TABLE product;")
cursor.execute("TRUNCATE TABLE client;")
cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

# --- PARTE 2: Ajustes Cirúrgicos (Garantindo os tamanhos) ---
cursor.execute("ALTER TABLE client MODIFY COLUMN Fname VARCHAR(50);")
cursor.execute("ALTER TABLE client MODIFY COLUMN Lname VARCHAR(50);")
cursor.execute("ALTER TABLE client MODIFY COLUMN Address VARCHAR(255);")
cursor.execute("ALTER TABLE product MODIFY COLUMN Pname VARCHAR(255) NOT NULL;")
cursor.execute("ALTER TABLE product MODIFY COLUMN avaliação VARCHAR(10) DEFAULT '0';")

db_setup.commit()
print("Banco limpo e estrutura ajustada. Iniciando inserção...")

# --- PARTE 3: Inserção de Dados ---

# 1. Clientes
cursor.execute("""
INSERT INTO client (Fname, Minit, Lname, CPF, Address) VALUES
('Maria', 'M', 'Silva', '12345678901', 'rua silva de prata 29, Carangola - Cidade das Flores'),
('Matheus', 'O', 'Pimentel', '98765432100', 'rua alameda 289, Centro - Cidade das Flores'),
('Ricardo', 'F', 'Silva', '45678912300', 'avenida alameda vinha 1009, Centro - Cidade das Flores'),
('Julia', 'S', 'França', '78912345600', 'rua laranjeiras 861, Centro - Cidade das Flores');
""")

# 2. Produtos
cursor.execute("""
INSERT INTO product (Pname, classification_kids, category, avaliação, size) VALUES
('Fone', false, 'Eletrônico', '4', NULL),
('Barbie', true, 'Brinquedos', '3', NULL),
('Body Carters', true, 'Vestuário', '5', NULL),
('Sofá Retrátil', false, 'Móveis', '3', '3x57x80'),
('Farinha', false, 'Alimentos', '2', NULL);
""")

# 3. Pedidos
cursor.execute("""
INSERT INTO orders (idOrderClient, orderStatus, orderDescription, sendValue, paymentCash) VALUES
(1, default, 'compra via aplicativo', NULL, 1),
(2, default, 'compra via aplicativo', 50, 0),
(3, 'Confirmado', NULL, NULL, NULL),
(4, default, 'compra via web', 150, 0);
""")

# 4. Pagamentos
cursor.execute("""
INSERT INTO payments (idClient, idPayment, typePayment, limitAvailable) VALUES
(1, 1, 'Cartão', 1000.00),
(2, 2, 'Boleto', NULL),
(3, 1, 'Cartão', 5000.00);
""")

db_setup.commit()
print("SUCESSO! Todos os dados foram inseridos sem conflitos.")

Banco limpo e estrutura ajustada. Iniciando inserção...
SUCESSO! Todos os dados foram inseridos sem conflitos.


In [19]:
print("--- RELATÓRIO: PEDIDOS POR CLIENTE ---")
query = """
SELECT c.idClient, Fname, COUNT(*) as Number_of_Orders
FROM client c
INNER JOIN orders o ON c.idClient = o.idOrderClient
GROUP BY c.idClient;
"""
cursor.execute(query)
results = cursor.fetchall()

for (id_c, name, count) in results:
    print(f"Cliente: {name} (ID: {id_c}) | Total de Pedidos: {count}")

print("\n--- RELATÓRIO: CLIENTES QUE NÃO COMPRARAM (LEFT JOIN) ---")
query_left = """
SELECT Fname, idOrder
FROM client
LEFT JOIN orders ON idClient = idOrderClient;
"""
cursor.execute(query_left)
for (name, order) in cursor.fetchall():
    status = "Sem pedidos" if order is None else f"Pedido ID: {order}"
    print(f"Cliente: {name.ljust(10)} | {status}")

--- RELATÓRIO: PEDIDOS POR CLIENTE ---
Cliente: Maria (ID: 1) | Total de Pedidos: 1
Cliente: Matheus (ID: 2) | Total de Pedidos: 1
Cliente: Ricardo (ID: 3) | Total de Pedidos: 1
Cliente: Julia (ID: 4) | Total de Pedidos: 1

--- RELATÓRIO: CLIENTES QUE NÃO COMPRARAM (LEFT JOIN) ---
Cliente: Maria      | Pedido ID: 1
Cliente: Matheus    | Pedido ID: 2
Cliente: Ricardo    | Pedido ID: 3
Cliente: Julia      | Pedido ID: 4


In [20]:
# 1. Criando a Tabela de Entrega (Requisito do Desafio)
cursor.execute("""
CREATE TABLE IF NOT EXISTS delivery (
    idDelivery INT AUTO_INCREMENT PRIMARY KEY,
    idDeliveryOrder INT,
    deliveryStatus ENUM('Em processamento', 'Postado', 'Em trânsito', 'Entregue') DEFAULT 'Em processamento',
    trackingCode VARCHAR(50) UNIQUE,
    CONSTRAINT fk_delivery_order FOREIGN KEY (idDeliveryOrder) REFERENCES orders(idOrder)
);
""")

# 2. Inserindo dados de teste para Entrega
cursor.execute("""
INSERT INTO delivery (idDeliveryOrder, deliveryStatus, trackingCode) VALUES
(1, 'Em trânsito', 'BR123456789TX'),
(2, 'Postado', 'BR987654321XP'),
(4, 'Em processamento', 'BR000111222ZZ');
""")

db_setup.commit()
print("Refinamento de 'Entrega' adicionado com sucesso!")

Refinamento de 'Entrega' adicionado com sucesso!


In [21]:
print("--- PERGUNTA 1: Algum vendedor também é fornecedor? ---")
# Comparando CNPJ de Seller e Supplier
query1 = """
SELECT s.SocialName as Nome, s.CNPJ
FROM seller s
INNER JOIN supplier sup ON s.CNPJ = sup.CNPJ;
"""
cursor.execute(query1)
res1 = cursor.fetchall()
print(res1 if res1 else "Nenhum vendedor é fornecedor no momento.")

print("\n--- PERGUNTA 2: Relação de produtos, fornecedores e estoques ---")
# Atributo derivado: Status de estoque baseado na quantidade
query2 = """
SELECT p.Pname as Produto, sup.SocialName as Fornecedor, ps.quantity as Qtd_Estoque,
       CASE
           WHEN ps.quantity > 50 THEN 'Estoque Alto'
           WHEN ps.quantity > 10 THEN 'Estoque Moderado'
           ELSE 'Necessita Reposição'
       END as Status_Nivel
FROM product p
INNER JOIN productSupplier psup ON p.idProduct = psup.idPsProduct
INNER JOIN supplier sup ON psup.idPsSupplier = sup.idSupplier
INNER JOIN storageLocation sl ON p.idProduct = sl.idLproduct
INNER JOIN productStorage ps ON sl.idLstorage = ps.idProdStorage;
"""
cursor.execute(query2)
for r in cursor.fetchall(): print(r)

print("\n--- PERGUNTA 3: Clientes que gastaram mais de R$ 50,00 em frete (HAVING) ---")
query3 = """
SELECT c.Fname, SUM(o.sendValue) as Total_Frete
FROM client c
JOIN orders o ON c.idClient = o.idOrderClient
GROUP BY c.idClient
HAVING Total_Frete > 50
ORDER BY Total_Frete DESC;
"""
cursor.execute(query3)
for r in cursor.fetchall(): print(r)

--- PERGUNTA 1: Algum vendedor também é fornecedor? ---
Nenhum vendedor é fornecedor no momento.

--- PERGUNTA 2: Relação de produtos, fornecedores e estoques ---

--- PERGUNTA 3: Clientes que gastaram mais de R$ 50,00 em frete (HAVING) ---
('Julia', 150.0)


In [22]:
# 1. Adicionando a coluna de Tipo (PF ou PJ) e a coluna de CNPJ
cursor.execute("ALTER TABLE client ADD COLUMN type_client ENUM('PF', 'PJ') DEFAULT 'PF';")
cursor.execute("ALTER TABLE client ADD COLUMN CNPJ CHAR(15);")

# 2. Tornando o CPF opcional (NULL), pois se for empresa (PJ), o CPF ficará vazio
cursor.execute("ALTER TABLE client MODIFY COLUMN CPF CHAR(11) NULL;")

# 3. Criando uma Unique Constraint para o CNPJ
cursor.execute("ALTER TABLE client ADD CONSTRAINT unique_cnpj_client UNIQUE (CNPJ);")

# 4. Inserindo um exemplo de cliente PJ para validar o refinamento
cursor.execute("""
INSERT INTO client (Fname, Lname, CPF, CNPJ, Address, type_client) VALUES
('Tech Store', 'LTDA', NULL, '123456789012345', 'Avenida Industrial 500, São Paulo', 'PJ');
""")

db_setup.commit()
print("REFINAMENTO CONCLUÍDO: A tabela 'client' agora suporta PF e PJ separadamente.")

# --- Validação Final ---
print("\n--- Lista Atualizada de Clientes (PF e PJ) ---")
cursor.execute("SELECT Fname, type_client, CPF, CNPJ FROM client;")
for r in cursor.fetchall():
    print(r)

REFINAMENTO CONCLUÍDO: A tabela 'client' agora suporta PF e PJ separadamente.

--- Lista Atualizada de Clientes (PF e PJ) ---
('Maria', 'PF', '12345678901', None)
('Matheus', 'PF', '98765432100', None)
('Ricardo', 'PF', '45678912300', None)
('Julia', 'PF', '78912345600', None)
('Tech Store', 'PJ', None, '123456789012345')
